In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import transforms3d as tf3d
import matplotlib
import scipy.signal
import time
from numba import jit
from scipy.integrate import odeint
from scipy.interpolate import interp1d
import os


In [2]:
try:
    os.makedirs(os.path.join("results","speed"))
except:
    pass

In [3]:
n_propellers=6


mass=369 #batterie
mass+=1640-114 #corps-carton
mass/=1e3
Area=np.pi*(11.0e-02)**2
r0=11e-02
rho0=1.204
kv_motor=800.0
pwmmin=1075.0
pwmmax=1950.0
U_batt=16.8

AN=n_propellers*Area
b10=14.44

In [4]:
# %%   ####### IMPORT DATA 
print("LOADING DATA...")


log_path="./logs/target/log_real_processed.csv"

raw_data=pd.read_csv(log_path)

print("PROCESSING DATA...")


prep_data=raw_data.drop(columns=[i for i in raw_data.keys() if (("forces" in i ) or ('pos' in i) or ("joy" in i)) ])
prep_data=prep_data.drop(columns=[i for i in raw_data.keys() if (("level" in i ) or ('Unnamed' in i) or ("index" in i)) ])
# print(prep_data)

tmin,tmax=-1,1e10

    
prep_data=prep_data[prep_data['t']>tmin]
prep_data=prep_data[prep_data['t']<tmax]
prep_data=prep_data.reset_index()
for i in range(3):
    prep_data['speed_pred[%i]'%(i)]=np.r_[prep_data['speed[%i]'%(i)].values[1:len(prep_data)],0]
    
    

prep_data['dt']=np.r_[prep_data['t'].values[1:]-prep_data['t'].values[:-1],0]
prep_data['t']-=prep_data['t'][0]
prep_data=prep_data.drop(index=[0,len(prep_data)-1])


for i in range(n_propellers):
    prep_data['omega_c[%i]'%(i+1)]=(prep_data['PWM_motor[%i]'%(i+1)]-pwmmin)/(pwmmax-pwmmin)*U_batt*kv_motor*2*np.pi/60

R_array=np.array([tf3d.quaternions.quat2mat([i,j,k,l]) for i,j,k,l in zip(prep_data['q[0]'],
                                                                          prep_data['q[1]'],
                                                                          prep_data['q[2]'],
                                                                          prep_data['q[3]'])])

vwx_body_dir=np.array([i.T@np.array([[1.0],[0],[0]]) for i in R_array] )
vwy_body_dir=np.array([i.T@np.array([[0.0],[1.0],[0]]) for i in R_array] )


prep_data['vwx_body_dir']=[i for i in vwx_body_dir.reshape((-1,3))]
prep_data['vwy_body_dir']=[i for i in vwy_body_dir.reshape((-1,3))]
prep_data['vwx_body_dir']=[i for i in vwx_body_dir.reshape((-1,3))]
prep_data['R']=[i for i in R_array]

LOADING DATA...
PROCESSING DATA...


In [5]:
# %% Speed identification

def model_forces(ct1,ct2,
               ch1,ch2,
               drag_coeffs=np.zeros(3),
                va_body_pred=np.zeros(3),
                 omegas=np.zeros(6),
               ct3=0,
              vanilla=False):
    
    vai,vaj,vak=va_body_pred

    T=sum(compute_single_motor_thrust_(omegas,
         vai,vaj,vak,
                    ct1,ct2,ct3,vanilla_test=vanilla))
    
    H=sum(compute_H(omegas,
          vai,vaj,vak,ch1,ch2,
                     ct1,ct2,ct3,vanilla_test=vanilla))
    
    T_vect=np.array([0,0,T])
    H_vect=np.array([-vai*H,-vaj*H,0])
    
    absva=np.linalg.norm(va_body_pred)
    Fa=-rho0*AN*absva*np.diag(drag_coeffs)@va_body_pred

    
    return -T_vect+H_vect+Fa


    


def predict_speed(ct1,
                   ct2,
                   ch1,
                   ch2,
                   di=0,
                   dj=0,
                   dk=0,
                   df=prep_data,
                   kt=0.0,
                   ct3=0,
                   vwi=0,
                   vwj=0,
                  vanilla=False,
                   dyn_motors=False):
    
    pred_speed_ned=np.zeros((len(df),3))
    pred_speed_ned[0]=np.array([df['speed[0]'].values[0],df['speed[1]'].values[0],df['speed[2]'].values[0]])
    
    omegas = compute_omegas(df,kt).reshape((len(df),6)) if dyn_motors else np.array([df['omega_c[%i]'%(i+1)] for i in range(n_propellers)]).T
    
    vw_body=vwi*np.array([i for i in df['vwx_body_dir'].values])+vwj*np.array([i for i in df['vwy_body_dir'].values])
    drag_coeffs=np.array([di,dj,dk])
                   
    
    for i in range(len(df)-1):
        R_temp=np.array(df['R'].values[i])
        v_body_pred= R_temp.T@(pred_speed_ned[i])
        
        forces=model_forces(ct1,ct2,ch1,ch2,
               drag_coeffs=drag_coeffs,
                va_body_pred=v_body_pred-vw_body[i],
                 omegas=omegas[i],
               ct3=ct3,
              vanilla=vanilla)
        

        acc_ned= R_temp@forces/mass + np.array([0,0,9.81])
        
        pred_speed_ned[i+1] = pred_speed_ned[i]+min(2e-2,df['dt'].values[i])*acc_ned
        
    return pred_speed_ned


def cost_global_speed_(X,option_booleans,df=prep_data,rms_score=False):
    
    ct1,ct2,ch1,ch2,di,dj,dk,ct3,kt,vwi,vwj = X

    motor_dyn , vanilla , with_c3 , dij , with_wind = option_booleans

    ct3 = ct3 if with_c3 else 0
    vwi,vwj=(vwi,vwj) if with_wind else (0,0) 
    dj = di if dij else dj
          
          
    Y=predict_speed(ct1,
                   ct2,
                   ch1,
                   ch2,
                   di=di,
                   dj=dj,
                   dk=dk,
                   df=df,
                   kt=kt,
                   ct3=ct3,
                   vwi=vwi,
                   vwj=vwj,
                  vanilla=vanilla,
                   dyn_motors=motor_dyn)
          
    ci=np.mean((Y[:,0]-df['speed[0]'])**2,axis=0)
    cj=np.mean((Y[:,1]-df['speed[1]'])**2,axis=0)
    ck=np.mean((Y[:,2]-df['speed[2]'])**2,axis=0)
    if rms_score==False:
        ci/=max(abs(df['speed[0]']))**2
        cj/=max(abs(df['speed[1]']))**2
        ck/=max(abs(df['speed[2]']))**2
        
        
        
        c=ci+cj+ck
#         print("ct1:%f ct2:%f ch1:%f ch2:%f di:%f dj:%f dk:%f ct3:%f kt:%f vwi:%f vwj:%f cost:%f"%(ct1,ct2,ch1,ch2,di,dj,dk,ct3,kt,vwi,vwj,c))

        return c

    else:
        score_rms_total=np.mean(np.linalg.norm(Y-np.c_[df['speed[0]'],df['speed[1]'],df['speed[2]']],axis=1))
        return np.sqrt([score_rms_total,ci,cj,ck])

    

## Run optim over whole dataset

In [6]:
# %% Global Optimization ACC

def compute_single_motor_thrust_(omega,vai,vaj,vak,c1,c2=0,c3=0,vanilla_test=False):
    
    if vanilla_test:
        T=rho0*Area*r0**2*omega**2*c1
        return T
    else:
        eta=vak/2-r0*omega*c2/4
        eta=eta+0.5*np.sqrt((vak+0.5*r0*omega*c2)**2
                            +2*c1*r0**2*omega**2+2*c3*(vai**2+vaj**2))

        T=rho0*Area*r0*omega*(c1*r0*omega-c2*(eta-vak))+rho0*Area*c3*(vai**2+vaj**2)
        return T

def compute_omegas(df,kt):
    # omegas is shape n timesteps x p actuators 
    omegas=np.array([df['omega_c[%i]'%(i+1)] for i in range(n_propellers)]).T
    
    b, a = scipy.signal.butter(1, 1./(2*np.pi)*kt,analog= False,fs=1./2.0/df['dt'].mean())
    zi = scipy.signal.lfilter_zi(b, a)
    new_omegas, _ =scipy.signal.lfilter(b, a, omegas,axis=0,zi=zi*omegas[0].reshape((1,-1)))
    return new_omegas


def compute_H(omega,vai,vaj,vak,ch1,ch2,c1=0,c2=0,c3=0,vanilla_test=False):
    if vanilla_test:
        return np.zeros(omega.shape)
    else:
        eta=vak/2-r0*omega*c2/4
        eta=eta+0.5*np.sqrt((vak+0.5*r0*omega*c2)**2
                            +2*c1*r0**2*omega**2+2*c3*(vai**2+vaj**2))
        H=rho0*Area*(ch1*r0*omega-ch2*(eta-vak))
        return H

def compute_acc_identification_globale(ct1,
                                       ct2,
                                       ch1,
                                       ch2,
                                       di=0,
                                       dj=0,
                                       dk=0,
                                       df=prep_data,
                                       kt=0.0,
                                       ct3=0,
                                       vwi=0,
                                       vwj=0,
                                      vanilla=False,
                                       dyn_motors=False):
    
    vw_body=vwi*np.array([i for i in df['vwx_body_dir'].values])+vwj*np.array([i for i in df['vwy_body_dir'].values])
    
    
    vai=df["speed_body[0]"]-vw_body[:,0]
    vaj=df["speed_body[1]"]-vw_body[:,1]
    vak=df["speed_body[2]"]-vw_body[:,2]
    
    gammai=df["gamma[0]"]
    gammaj=df["gamma[1]"]
    gammak=df["gamma[2]"]
    
    omegas = compute_omegas(df,kt).reshape((len(df),6)) if dyn_motors else np.array([df['omega_c[%i]'%(i+1)] for i in range(n_propellers)]).T
    
    
    T=sum([compute_single_motor_thrust_(omegas[:,i],
         vai,vaj,vak,
                    ct1,ct2,ct3,vanilla_test=vanilla) for i in range(n_propellers)])
    
    H=sum([compute_H(omegas[:,i],
          vai,vaj,vak,ch1,ch2,
                     ct1,ct2,ct3,vanilla_test=vanilla) for i in range(n_propellers)])
    
    H_vect=np.c_[-vai*H,-vaj*H,np.zeros(H.shape)]
    T_vect=np.c_[np.zeros(T.shape),np.zeros(T.shape),T]
    
    absva=np.sqrt(vai**2+vaj**2+vak**2)
    Fa=-rho0*AN*np.c_[di*absva*vai,dj*absva*vaj,dk*absva*vak]

    return -T_vect/mass+H_vect/mass+np.c_[gammai,gammaj,gammak]+Fa/mass


    
    

## Run optim over batched dataset

In [7]:
def genbatches(dataframe,nsecs):
        
    "if nsecs = all, all the dataset is used"
    "else, optim is performed on a sample corresponding to nsecs seconds of flight"
    
    if nsecs=='all':
        return [dataframe]
    else:
        N_minibatches=round(dataframe["t"].max()/nsecs) # 22 for flight 1, ?? for flight 2
        return np.array_split(dataframe, N_minibatches)
    

In [8]:
# %% global speed opti: parallel run with batches


headers_df = ['motor_dyn,vanilla,with_c3,dij,with_wind'.replace(' ','').split(',')]
headers_df = headers_df + ['ct1,ct2,ch1,ch2,di,dj,dk,ct3,kt,vwi,vwj'.replace(' ','').split(',')]
headers_df = headers_df + ['score_opti,score_rms,score_x,score_y,score_z,duration,nsecs,nbatch'.replace(' ','').split(',')]
headers_df = [item for sublist in headers_df for item in sublist]


params_batches = []

for motor_dyn in [False]:
    for vanilla in [False]:
        for with_c3 in [True]:
            for dij in [True]:
                for with_wind in [True]:
                    for batch_size_in_secs in [0.2,0.5,1.0]:
                        if not( vanilla and with_c3):
                            params_batches.append([motor_dyn , vanilla , with_c3 , dij , with_wind ,batch_size_in_secs]) 
                        
print(len(params_batches)," param sets")              

#     ct1,ct2,ch1,ch2,di,dj,dk,ct3,kt,vwi,vwj = X
bnds=[(0,None),
      (None,None),
      (0,None),
      (None,None),
      (0,None),
      (0,None),
      (0,None),
      (None,None),
      (1,15),
      (-8,8),
      (-8,8)]


X0_id_globale=np.array([0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0, #c3
                        5.0, #kt
                        0, #vwi
                        0]) #vwj


from multiprocessing import Pool

def run_in_process(params_el):
    
    batch_size_in_secs=params_el[-1]
    
    batches=genbatches(prep_data,batch_size_in_secs)
    
    for i,df in enumerate(batches):

        speed_data_list=[]

        t0=time.time()

#         print(i,"/", len(batches)," begin optim")
        
        sol_= minimize(cost_global_speed_,X0_id_globale,bounds=bnds,args=(params_el[:-1],df))
        
#         print(i,"/", len(batches)," computing scores ")

        score_rms,score_x,score_y,score_z = cost_global_speed_(sol_['x'],params_el[:-1],prep_data,rms_score=True)
#         print('here?')
            
        optim_duration = time.time()-t0

        speed_data_list.append(np.r_[params_el[:-1],sol_['x'],sol_['fun'],
                                     score_rms,score_x,score_y,score_z,optim_duration,
                                    batch_size_in_secs,i])
#         print(speed_data_list)
        df_res =  pd.DataFrame(data=speed_data_list,columns=headers_df)   
#         print(df_res)
        df_res.to_csv('./results/speed/speed_'+str(time.time())+str(round(10000*np.random.random()))+".csv")
#         print(i,"/", len(batches),"  ", params_el, "DONE") 


if __name__ == '__main__':

    pool = Pool(processes=2)
    pool.map(run_in_process, params_batches)
    

3  param sets


/tmp/ipykernel_13601/2957862452.py:10: RuntimeWarning: invalid value encountered in sqrt
  eta=eta+0.5*np.sqrt((vak+0.5*r0*omega*c2)**2
/tmp/ipykernel_13601/2957862452.py:31: RuntimeWarning: invalid value encountered in sqrt
  eta=eta+0.5*np.sqrt((vak+0.5*r0*omega*c2)**2
/tmp/ipykernel_13601/2957862452.py:10: RuntimeWarning: overflow encountered in square
  eta=eta+0.5*np.sqrt((vak+0.5*r0*omega*c2)**2
/tmp/ipykernel_13601/2957862452.py:11: RuntimeWarning: overflow encountered in double_scalars
  +2*c1*r0**2*omega**2+2*c3*(vai**2+vaj**2))
/tmp/ipykernel_13601/2957862452.py:13: RuntimeWarning: invalid value encountered in multiply
  T=rho0*Area*r0*omega*(c1*r0*omega-c2*(eta-vak))+rho0*Area*c3*(vai**2+vaj**2)
/tmp/ipykernel_13601/2957862452.py:13: RuntimeWarning: overflow encountered in double_scalars
  T=rho0*Area*r0*omega*(c1*r0*omega-c2*(eta-vak))+rho0*Area*c3*(vai**2+vaj**2)
/tmp/ipykernel_13601/2957862452.py:31: RuntimeWarning: overflow encountered in square
  eta=eta+0.5*np.sqrt((va

KeyboardInterrupt: 